<a href="https://colab.research.google.com/github/simulate111/General/blob/main/OSLO_Norway_NASAPower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#pip install meteostat

In [36]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
# Oslo, Norway
LAT = 59.9139
LON = 10.7522

# Date Range (YYYYMMDD)
START_DATE = "20240101"
END_DATE = "20241231"

# API Endpoint
base_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"

# Parameters:
# T2M = Temp (2m), WS10M = Wind (10m)
# ALLSKY_SFC_SW_DWN = Global Horizontal Irradiance (GHI)
# ALLSKY_SFC_SW_DNI = Direct Normal Irradiance (DNI)
# ALLSKY_SFC_SW_DIFF = Diffuse Horizontal Irradiance (DHI)
params = {
    "parameters": "T2M,WS10M,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF",
    "community": "RE", # Renewable Energy
    "longitude": LON,
    "latitude": LAT,
    "start": START_DATE,
    "end": END_DATE,
    "format": "JSON"
}

print(f"Fetching NASA POWER Data for Oslo ({START_DATE}-{END_DATE})...")

try:
    r = requests.get(base_url, params=params)

    if r.status_code == 200:
        data = r.json()

        # Parse nested JSON response
        properties = data['properties']['parameter']
        timestamps = sorted(properties['T2M'].keys())

        records = []
        for ts in timestamps:
            records.append({
                'Time_Str': ts,
                'Temperature_C': properties['T2M'].get(ts),
                'Wind_Speed_ms': properties['WS10M'].get(ts),
                'GHI_Wm2': properties['ALLSKY_SFC_SW_DWN'].get(ts),
                'DNI_Wm2': properties['ALLSKY_SFC_SW_DNI'].get(ts),
                'DHI_Wm2': properties['ALLSKY_SFC_SW_DIFF'].get(ts)
            })

        df = pd.DataFrame(records)

        # Format Timestamps
        df['Time'] = pd.to_datetime(df['Time_Str'], format='%Y%m%d%H')
        df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
        df['Hour'] = df['Time'].dt.strftime('%H:%M')

        # Clean Missing Data (-999 -> NaN)
        df = df.replace(-999, pd.NA)

        # Select Columns
        cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2', 'DNI_Wm2', 'DHI_Wm2']
        df = df[cols]

        print("-" * 30)
        print(df.head())
        print("-" * 30)

        filename = "Oslo_NASA_POWER_2024.csv"
        df.to_csv(filename, index=False)
        print(f"Success! Saved to {filename}")

    else:
        print(f"Error {r.status_code}: {r.text}")

except Exception as e:
    print(f"Connection failed: {e}")

Fetching NASA POWER Data for Oslo (20240101-20241231)...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms  GHI_Wm2  DNI_Wm2  DHI_Wm2
0  2024-01-01  00:00          -7.21           3.37      0.0      0.0      0.0
1  2024-01-01  01:00          -7.47           3.33      0.0      0.0      0.0
2  2024-01-01  02:00          -7.73           3.29      0.0      0.0      0.0
3  2024-01-01  03:00          -7.91           3.27      0.0      0.0      0.0
4  2024-01-01  04:00          -7.96           3.22      0.0      0.0      0.0
------------------------------
Success! Saved to Oslo_NASA_POWER_2024.csv
